In [ ]:
## code was written using Python 2.7.13 ##
import warnings
warnings.filterwarnings('ignore')
import numpy as np #v1.14.4
np.set_printoptions(threshold=np.nan)
import matplotlib.pyplot as plt #v1.5.1
%matplotlib inline
import pandas as pd #0.23.0
import sklearn #0.18.1
from sklearn import svm
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV

In [ ]:
%cd 'C:\Users\Stuber Lab\Desktop\Raw data\obesity slice ephys'
data=pd.read_excel('Excitability summary.xlsx')
summarydata=pd.DataFrame(data)
headers=list(summarydata.columns.values)
summarydata=summarydata.replace(['eYFP','none'],[1,0])
summarydata=summarydata.replace(['Control','High Fat'],[0,1])

In [ ]:
##Use SVM to classify data ###
def ephys_svm(filename,predict,params,iterations,test_data):
    parameters = [{'C': [0.001, .01,.1,1,10,100,1000], 'kernel': ['linear']},
                  {'C': [0.001, .01,.1,1,10,100,1000], 'gamma': [0.001, .01,.1,1,10,100,1000], 'kernel': ['rbf']}]  
    df=summarydata[summarydata.label != label_tag]
    combined=params+predict
    df=df[combined].dropna()
    pred_score=np.nan*np.zeros((iterations))
    rand_score=np.nan*np.zeros((iterations))
    for i in range(iterations):
        print '\nRun #: ', i+1
        ## if splits result in insufficient data to classify, skip split and move on ##
        try:
            ##randomly subset df into test and train sets
            msk = np.random.rand(len(df))>test_data
            train = df[msk]
            ##use gridsearch to select optimal parameters for each run
            clf = GridSearchCV(SVC(), parameters,cv=10)
            train_labels=train[predict].values
            c,r=train_labels.shape
            train_labels=train_labels.reshape(c,)
            clf.fit(train[params], train_labels)
            print 'prediction accuracy = ', clf.best_score_
            pred_score[i]=(clf.best_score_)*100
            ##random array of 1s and 0s for testing classifier
            rand_group=pd.DataFrame(np.random.randint(2,size=train.shape[0]))
            clf.fit(train[params], rand_group[0])
            print 'shuffled accuracy = ', clf.best_score_
            rand_score[i]=(clf.best_score_)*100
        except Exception:
            pass
    plt.figure(figsize=(5,5))
    ax=plt.subplot(111)
    n = np.arange(1,len(pred_score)+1) / np.float(len(pred_score))
    Xs = np.sort(pred_score)
    plt.step(Xs,n,color='g',label='Model prediction',alpha=0.7,linewidth=2)
    n2= np.arange(1,len(rand_score)+1) / np.float(len(rand_score))
    Xs2= np.sort(rand_score)
    plt.step(Xs2,n,color='r',label='Random',alpha=0.7,linewidth=2)
    ax.set_axis_bgcolor('white')
    plt.xlabel('Decoding accuracy %',fontsize=14)
    plt.ylabel('Proportion of runs',fontsize=14)
    plt.legend(fontsize=14,loc=2)
    plt.savefig(filename+str(iterations)+str('iter_')+str(test_data)+'test_data_prediction_accuracy_CDF.pdf')
    np.savetxt(filename+str(iterations)+str('iter_')+str(test_data)+'_prediction_accuracy.csv',pred_score,delimiter=',')
    np.savetxt(filename+str(iterations)+str('iter_')+str(test_data)+'_RANDOM_prediction_accuracy.csv',rand_score,delimiter=',')
    plt.show()
    print "mean decoding accuracy = ", np.nanmean(pred_score)
    print "mean RANDOM decoding accuracy = ",np.nanmean(rand_score)
#     return pred_score,rand_score

In [ ]:
### Run SVM for eYFP+ cells to predict group based on rheo and max latency ###
label_tag=0  ## Choose 0 for eYFP+ or 1 for eYFP- ##
filename='test_function_YFP+'
predict=['group'] ## group=high fat/control
params=['rheo','max latency'] ## choose params to use for classification 
iterations=1000
test_data=.1  #defines split for test/train sets
#####

ephys_svm(filename,predict,params,iterations,test_data)

In [ ]:
### Run SVM for eYFP- cells to predict group based on rheo and max latency ###
label_tag=1  ## Choose 0 for eYFP+ or 1 for eYFP- ##
filename='test_function_YFP-'
predict=['group'] ## group=high fat/control
params=['rheo','max latency'] ## choose params to use for classification 
iterations=1000
test_data=.1  #defines split for test/train sets
#####

ephys_svm(filename,predict,params,iterations,test_data)